## Using the Facebook API

Before you can use the api you need to set up a developer account with facebook
https://developers.facebook.com


https://towardsdatascience.com/how-to-use-facebook-graph-api-and-extract-data-using-python-1839e19d6999

- **Getting the Access Token:**
- To be able to extract data from Facebook using a python code you need to register as a developer on Facebook and then have an access token. Here are the steps for it.
- Go to link developers.facebook.com, create an account there.
- Go to link developers.facebook.com/tools/explorer.
- Go to “My apps” drop down in the top right corner and select “add a new app”. Choose a display name and a category and then “Create App ID”.
- Again get back to the same link developers.facebook.com/tools/explorer. You will see “Graph API Explorer” below “My Apps” in the top right corner. From “Graph API Explorer” drop down, select your app.
- Then, select “Get Token”. From this drop down, select “Get User Access Token”. Select permissions from the menu that appears and then select “Get Access Token.”
- Go to link developers.facebook.com/tools/accesstoken. Select “Debug” corresponding to “User Token”. Go to “Extend Token Access”. This will ensure that your token does not expire every two hours.


## Imports

In [1]:
import urllib3
import os
import time
import facebook
from facebook import GraphAPIError
import requests
import json
import numpy as np
import pandas as pd
import re

In [2]:
# User Access Token from FB developer account
token = 'EAADe3adNZCDABADFJGuU1ZB4BFqxE5HZCBOqZCc4pu5QIh0n6xBP1xrb3QsQ0qVU8jMhSnH9zZArCSydzEpHc6CH0YtFHiHT2jePA1fa9YNyK86Qt5ZAYSPHDNZAZAZCPFWiZABRzRZBqO3APKamy1KVNK1mEsGcjCDHDWtJS9ZALxo0R2zg2nheHz0i'


## Functions 

In [77]:
def fb_pagefinder(activity, center, search_radius, token=token):
    lat = center[0]
    lng = center[1]
    graph = facebook.GraphAPI(access_token=token, timeout=200)
    events = graph.request(
        'search?q=events%20{}&fields=name,id,description,location,member_count&type=place&center={},{}&distance={}&limit=100'.format(
        activity, lat, lng, search_radius))
    return events
# def fb_pagefinder(activity, center, search_radius, token=token):
#     lat = format(center[0], '.10g')
#     lng = format(center[1], '.10g')
#     print(lat)
#     print(lng)
#     graph = facebook.GraphAPI(access_token=token, timeout=200)
#     events = graph.request(
#         'search?q=events%20{}&fields=name,id,description,member_count,privacy,'
#         +'member_request_count,location&type=place&center={},{}&distance={}&limit=1000'.format(
#         activity, lat, lng, search_radius))
#     return events


def get_nextpg(events, activity):
    all_events=[]
    counter = 0
    
    while(True):
        try:
            for event in events:
                counter += 1
                all_events.append(events['data'])
            # Attempt to make a request to the next page of data, if it exists.
            events=requests.get(events['paging']['next']).json()
            print(str(counter)+'pages')
        except KeyError:
            # When there are no more pages (['paging']['next']), break from the
            # loop and end the script.
            break
            
    new_npy = np.load('facebook_data/fb_%s.npy'%activity) if os.path.isfile('facebook_data/fb_%s.npy'%activity) else [] #get data if exist
    np.save('facebook_data/fb_%s.npy'%activity,np.append(new_npy,all_events)) #save the new
    
    return all_events


        
def fb_main(activity, center, search_radius, token=token):
    events = fb_pagefinder(activity, center, search_radius, token=token)
    get_nextpg(events, activity)
    

    
def get_df(activity):
    message= "--(fb_%s.npy) doesn't exist--"%activity
    data= 'facebook_data/fb_%s.npy'%activity
    fb_data = np.load(data) if os.path.isfile(data) else print(message)
    
    if len(fb_data) != 0:
        try:
            fb_info =fb_data[0]['data']
        except KeyError:
            fb_info = fb_data

        grp_id,name,location,latitude,longitude, member_count, member_request_count, privacy=[],[],[],[],[],[],[],[]
        for i in fb_info:
            grp_id.append(i['id'])
            name.append(i['name'])
            location.append(i['location'])
            member_count.append(i['member_count'])
            member_request_count.append(i['member_request_count'])
            privacy.append(i['privacy'])  


            
        df = pd.DataFrame([grp_id,name,location,member_count, member_request_count, privacy]).T
        df.columns=['grp_id','name','location', 'member_count', 'member_request_count', 'privacy']
        
        return df
    
    else:
        print('--no data---')
        pass

## Data

In [4]:
data = pd.read_csv('Loneliness group key words.csv')
data.head()

,Children (< 16 years),Young adults (16 - 24 years),Adults (25 - 49 years),Older people (> 50 years),Welsh translation,Polish translation
0,Sport,Sport,Sport,Sport,Chwaraeon,Sport
1,NaN,Yoga,Yoga,Yoga,Yoga,Joga
2,NaN,Bokwa,Bokwa,Bokwa,Bokwa,Bokwa
3,NaN,Fitness,Fitness,Fitness,Ffitrwydd,Zdatność
4,Gym,Gym,Gym,Gym,Campfa,Siłownia


### Concatenating all the different colmuns:
* this piece of code drops all the duplicates, drops all NA values and sorts them in alphabetical order

In [5]:
# all Groups in the English Language
data_1 = pd.concat([data['Children (< 16 years)'],
                   data['Young adults (16 - 24 years)'],
                   data['Adults (25 - 49 years)'],
                   data['Older people (> 50 years)']
                  ]).drop_duplicates().dropna().sort_values().reset_index(drop=True)

data_2 = pd.concat([data['Welsh translation'],
                   data['Polish translation']
                  ]).drop_duplicates().dropna().sort_values().reset_index(drop=True)

In [6]:
data_1[:10]

0            10km
1             2km
2             5km
3       Allotment
4             Art
5         Beavers
6            Bike
7            Bird
8    Body Balance
9       Body Pump
dtype: object

In [78]:
cities = np.load('main_cities.npy')

In [79]:
cities[:10]

array([[ 51.89107295,  -0.42303041],
       [ 51.456659  ,  -0.9696512 ],
       [ 52.0429797 ,  -0.7589607 ],
       [ 52.5738875 ,  -0.2416701 ],
       [ 53.4018582 ,  -2.56802236],
       [ 50.338466  ,  -4.7882104 ],
       [ 54.53525645,  -1.55294298],
       [ 54.8948478 ,  -2.9362311 ],
       [ 52.9212617 ,  -1.4761491 ],
       [ 50.3712659 ,  -4.1425658 ]])

In [63]:
format(cities[9][0], '.10g')

str

In [43]:
str1 = str(cities[0])
txt1 = re.findall(r"[-+]?\d*\.\d+|\d+ ", str1)

In [44]:
str(txt1)

"['51.89107295', '-0.42303041']"

In [45]:
txt2 = re.findall('(\d+.\d+)', str(str1))

In [48]:
(txt2[0])

'51.89107295'

#### the waits:  facebook has a limit of 200 requests per hour hence the waits

In [ ]:
counter = 0

for d in data_1:
    print(d)
    
    for city in cities:
        counter += 1
        time.sleep(18)
        try:
            fb_main(d,city,16093.4)
        except GraphAPIError as e:
            err1 = 'Please retry your request later'
            err2 = 'request limit reached'
            err3 = 'exceeded the rate limit'
            
            if err1 in str(e):
                print(e)
                time.sleep(600)
                continue
                
            elif err2 in str(e):
                print(e)
                time.sleep(600)
                continue
                
            elif err3 in str(e):
                print(e)
                time.sleep(3600)
                continue
            else:
                print(city)
                raise e
                
        print('--%s--counter--%d--'%(city,counter))

10km
--[ 51.89107295  -0.42303041]--counter--1--
--[ 51.456659   -0.9696512]--counter--2--
--[ 52.0429797  -0.7589607]--counter--3--
--[ 52.5738875  -0.2416701]--counter--4--
--[ 53.4018582   -2.56802236]--counter--5--
--[ 50.338466   -4.7882104]--counter--6--
--[ 54.53525645  -1.55294298]--counter--7--
--[ 54.8948478  -2.9362311]--counter--8--
--[ 52.9212617  -1.4761491]--counter--9--
--[ 50.3712659  -4.1425658]--counter--10--
--[ 50.74467035  -1.8579809 ]--counter--11--
--[ 53.7435722  -0.3394758]--counter--12--
--[ 50.84522125  -0.14975851]--counter--13--
--[ 51.6020246  -0.9153487]--counter--14--
--[ 51.8653705  -2.2458192]--counter--15--
--[ 51.5073219  -0.1276474]--counter--16--
--[ 53.4791301  -2.2441009]--counter--17--
--[ 50.9025349  -1.404189 ]--counter--18--
--[ 52.0553813  -2.7151735]--counter--19--
--[ 51.6553875  -0.3957425]--counter--20--
--[ 50.72414995  -1.17274588]--counter--21--
--[ 51.3876562   0.5457712]--counter--22--
--[ 53.8182212  -3.0564845]--counter--23--
--[

In [70]:
np.load('facebook_data/fb_10km.npy')

array([ {'name': 'The Hitchin 10km', 'id': '739223372954229', 'location': {'city': 'Hitchin', 'country': 'United Kingdom', 'latitude': 51.94729, 'longitude': -0.27851, 'street': 'Market Place'}},
       {'name': 'The Hitchin 10km', 'id': '739223372954229', 'location': {'city': 'Hitchin', 'country': 'United Kingdom', 'latitude': 51.94729, 'longitude': -0.27851, 'street': 'Market Place'}}], dtype=object)